In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession
        .builder
        .appName("PythonMnMCount")
        .getOrCreate())

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# Leemos los datos y quitamos filas con NA y convertimos a numéricas las columnas inferidas incorrectamente
flightsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("C:/Users/laura.serrano/Desktop/flights-jan-apr-2018.csv")

# Convertimos a enteros y re-categorizamos ArrDelay en una nueva columna ArrDelayCat
# None (< 15 min), Slight(entre 15 y 60 min), Huge (> 60 min)

cleanFlightsDF = flightsDF.withColumn("ArrDelayCat", F.when(F.col("ArrDelay") < 15, "None")\
                                                      .when((F.col("ArrDelay") >= 15) & (F.col("ArrDelay") < 60), "Slight")\
                                                      .otherwise("Huge"))\
                           .cache()

In [4]:
cleanFlightsDF.show()

+-----+----------+---------+----------+------+------------+----+------------+-------+--------+-------+--------+---------+----------------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+-----------+
|Month|DayofMonth|DayOfWeek|FlightDate|Origin|  OriginCity|Dest|    DestCity|DepTime|DepDelay|ArrTime|ArrDelay|Cancelled|CancellationCode|Diverted|ActualElapsedTime|AirTime|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|ArrDelayCat|
+-----+----------+---------+----------+------+------------+----+------------+-------+--------+-------+--------+---------+----------------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+-----------+
|    1|        14|        7|2018-01-14|   SYR|Syracuse, NY| DTW| Detroit, MI|   null|    null|   null|    null|      1.0|               B|     0.0|             null|   null|   374.0|        null|        null|    nu

In [ ]:
# PREGUNTA: ¿Cuáles son los vuelos (origen, destino) con mayor retraso medio? 
# ¿Cuántos vuelos existen entre cada par de aeropuertos?

#PISTA: Tras hacer las agregaciones para cada pareja "Origin", "Dest" 
#(una agregación para el retraso medio y otra para contar), 
#aplica el método sort(F.col("avgDelay").desc()) para ordenar de forma decreciente por la nueva columna del retraso medio.

In [10]:
from pyspark.sql.functions import col
vuelos_mayor_retraso_llegada = cleanFlightsDF.groupBy("Origin", "Dest")\
                                     .agg(F.mean(col("ArrDelay")).alias("avgDelay"),\
                                          F.count("*").alias("num_vuelos"))\
                                     .sort(F.col("avgDelay").desc())
vuelos_mayor_retraso_llegada.show()

+------+----+------------------+----------+
|Origin|Dest|          avgDelay|num_vuelos|
+------+----+------------------+----------+
|   RDM| MFR|            1347.0|         2|
|   MDT| HPN|             798.0|         1|
|   ORD| GTF|             212.0|         1|
|   ICT| DAY|             210.0|         1|
|   ELM| ATL|             169.0|         2|
|   DSM| PIA|             168.0|         1|
|   ERI| ITH|             160.0|         1|
|   YNG| PIE|             141.0|         1|
|   CMH| HOU|             120.0|         1|
|   HRL| DAL|             111.0|         1|
|   PPG| HNL|109.85714285714286|        35|
|   HNL| PPG|105.85714285714286|        35|
|   PIE| YNG|             104.0|         1|
|   AVP| SFB|              93.0|         1|
|   ACY| MSY| 87.45454545454545|        11|
|   CPR| LAS|              85.0|         1|
|   LAS| CPR|              82.0|         1|
|   TTN| BNA|              76.5|        10|
|   MSP| PVD|              74.0|         1|
|   TUL| OKC|              69.0|

In [11]:
#Redondear el retraso medio a un decimal

In [14]:
vuelos_mayor_retraso_llegada_round = cleanFlightsDF.groupBy("Origin", "Dest")\
                                     .agg(F.round(F.mean(col("ArrDelay")),1).alias("avgDelay"),\
                                          F.count("*").alias("num_vuelos"))\
                                     .sort(F.col("avgDelay").desc())
vuelos_mayor_retraso_llegada_round.show()

+------+----+--------+----------+
|Origin|Dest|avgDelay|num_vuelos|
+------+----+--------+----------+
|   RDM| MFR|  1347.0|         2|
|   MDT| HPN|   798.0|         1|
|   ORD| GTF|   212.0|         1|
|   ICT| DAY|   210.0|         1|
|   ELM| ATL|   169.0|         2|
|   DSM| PIA|   168.0|         1|
|   ERI| ITH|   160.0|         1|
|   YNG| PIE|   141.0|         1|
|   CMH| HOU|   120.0|         1|
|   HRL| DAL|   111.0|         1|
|   PPG| HNL|   109.9|        35|
|   HNL| PPG|   105.9|        35|
|   PIE| YNG|   104.0|         1|
|   AVP| SFB|    93.0|         1|
|   ACY| MSY|    87.5|        11|
|   CPR| LAS|    85.0|         1|
|   LAS| CPR|    82.0|         1|
|   TTN| BNA|    76.5|        10|
|   MSP| PVD|    74.0|         1|
|   TUL| OKC|    69.0|         1|
+------+----+--------+----------+
only showing top 20 rows



In [15]:
#Vuelos ordenados por categoria segun el retraso de llegada entre los países con codigo MCI y MKE

In [16]:
sortedAvgDelaysDF = cleanFlightsDF.groupBy("Origin", "Dest", "ArrDelayCat").agg(
                                        F.count("*").alias("cuantos"))\
                                    .where("Origin = 'MCI' and Dest = 'MKE'").show()

+------+----+-----------+-------+
|Origin|Dest|ArrDelayCat|cuantos|
+------+----+-----------+-------+
|   MCI| MKE|       Huge|      8|
|   MCI| MKE|       None|    172|
|   MCI| MKE|     Slight|     18|
+------+----+-----------+-------+



In [ ]:
#Mostrar de cada par origen-destino del cuál hay un vuelo, cuántos se retrasan a la salida agrupados por categoria

In [17]:
sortedAvgDelaysDF = cleanFlightsDF.groupBy("Origin", "Dest").pivot("ArrDelayCat").agg(
                                        F.count("*").alias("cuantos")
)
sortedAvgDelaysDF.show()

+------+----+----+----+------+
|Origin|Dest|Huge|None|Slight|
+------+----+----+----+------+
|   MCI| MKE|   8| 172|    18|
|   TPA| ACY|   4| 112|     4|
|   PBI| DCA|  37| 393|    49|
|   DSM| EWR|  10|  94|    14|
|   MDW| MEM|  22| 172|    42|
|   ORD| PDX|  23| 528|    85|
|   SHD| LWB|   2|  25|  null|
|   SMF| BUR|  61| 720|   124|
|   STS| PHX|   9| 105|    14|
|   MCI| IAH|  38| 487|    54|
|   FSD| ATL|   9|  83|     9|
|   PHL| MCO| 162|1291|   273|
|   ATL| GSP|  47|1080|   106|
|   SJC| LIH|   1|  83|     5|
|   DSM| MCO|   1|  30|    10|
|   IAD| ILM|   2|  34|     8|
|   PBG| PGD|   1|  19|     6|
|   LBB| DEN|  20| 184|    20|
|   SNA| PHX|  57| 967|   256|
|   PIE| AVP|null|   1|  null|
+------+----+----+----+------+
only showing top 20 rows



In [ ]:
#Mostrar de cada cada pais de origen del cuál hay un vuelo, cuántos se retrasan a la salida agrupados por categoria

In [18]:
cleanFlightsDF.groupBy("Origin")\
              .pivot("ArrDelayCat")\
              .count().show()


+------+----+-----+------+
|Origin|Huge| None|Slight|
+------+----+-----+------+
|   BGM|  32|  223|    27|
|   PSE|  26|  194|    27|
|   INL|  21|  172|    14|
|   PPG|   4|   21|    10|
|   MSY|1298|15297|  2262|
|   GEG| 207| 5836|   596|
|   SNA| 705|12038|  1464|
|   BUR| 637| 7025|  1253|
|   GRB| 173| 1441|   180|
|   GTF|  59|  731|    86|
|   IFP|null|   31|    14|
|   IDA|  32|  699|    61|
|   LWB|   5|   43|    10|
|   GRR| 667| 5068|   863|
|   PVU|  14|  133|    22|
|   JLN|  27|  197|    53|
|   EUG| 157| 2360|   261|
|   PSG|  12|  211|    17|
|   GSO| 528| 4011|   623|
|   MYR| 262| 2335|   290|
+------+----+-----+------+
only showing top 20 rows



In [ ]:
#PREGUNTA: ¿Es el avión un medio de transporte fiable? Mostrar el número de vuelos en cada categoría de retraso.

En lugar de llamar agg(F.count("*")), podemos llamar a la transformación count() sobre el resultado de groupBy(), y creará automáticamente una columna llamada "count" con los conteos para cada grupo.
Ahora agrupar también por cada aeropuerto de origen, y mostrando una columna distinta por cada tipo de retraso, con el recuento. PISTA: utilizar la función pivot("colName").

In [26]:
vuelos_retraso= (cleanFlightsDF.groupBy("Origin", "Dest")\
              .pivot("ArrDelayCat")\
              .count())
vuelos_retraso.show()

+------+----+----+----+------+
|Origin|Dest|Huge|None|Slight|
+------+----+----+----+------+
|   MCI| MKE|   8| 172|    18|
|   TPA| ACY|   4| 112|     4|
|   PBI| DCA|  37| 393|    49|
|   DSM| EWR|  10|  94|    14|
|   MDW| MEM|  22| 172|    42|
|   ORD| PDX|  23| 528|    85|
|   SHD| LWB|   2|  25|  null|
|   SMF| BUR|  61| 720|   124|
|   STS| PHX|   9| 105|    14|
|   MCI| IAH|  38| 487|    54|
|   FSD| ATL|   9|  83|     9|
|   PHL| MCO| 162|1291|   273|
|   ATL| GSP|  47|1080|   106|
|   SJC| LIH|   1|  83|     5|
|   DSM| MCO|   1|  30|    10|
|   IAD| ILM|   2|  34|     8|
|   PBG| PGD|   1|  19|     6|
|   LBB| DEN|  20| 184|    20|
|   SNA| PHX|  57| 967|   256|
|   PIE| AVP|null|   1|  null|
+------+----+----+----+------+
only showing top 20 rows



In [38]:
vuelo_huge=vuelos_retraso.agg(F.sum(col("Huge")).alias("total_huge"),\
                              F.sum(col("None")).alias("total_none"), \
                              F.sum(col("Slight")).alias("total_slight"))
vuelo_huge.show()

+----------+----------+------------+
|total_huge|total_none|total_slight|
+----------+----------+------------+
|    200152|   2004727|      298234|
+----------+----------+------------+



In [ ]:
#PREGUNTA: ¿Hay relación entre el día de la semana y el retraso a la salida o a la llegada?

#PISTA: Calcula el retraso medio a la salida y a la llegada para cada día de la semana y ordena por una de ellas descendentemente.


In [40]:
from pyspark.sql.functions import col
vuelos_dia_retraso = cleanFlightsDF.groupBy("DayOfWeek")\
                                     .agg(F.mean(col("ArrDelay")).alias("avgArrDelay"),\
                                          F.mean(col("DepDelay")).alias("avgDepDelay"))\
                                     .sort(F.col("avgArrDelay").desc())
vuelos_dia_retraso.show()

+---------+-------------------+------------------+
|DayOfWeek|        avgArrDelay|       avgDepDelay|
+---------+-------------------+------------------+
|        1|  5.391113068725289|10.430177708665964|
|        5|  5.027363815430113|10.220785437977693|
|        7| 3.2344449424598207| 9.142161259888235|
|        3| 3.0525338339576717|  8.47071347600168|
|        2| 2.8412409647873806| 8.246502522185226|
|        4| 2.7390527404801026|  8.35856546210902|
|        6|-0.5748593305876211| 6.278199328016013|
+---------+-------------------+------------------+



In [45]:
(cleanFlightsDF.select("ArrDelay", "DayOfWeek").where(col("ArrDelay")<0)).show()

+--------+---------+
|ArrDelay|DayOfWeek|
+--------+---------+
|   -13.0|        3|
|   -24.0|        7|
|   -10.0|        1|
|   -15.0|        2|
|   -24.0|        3|
|   -27.0|        4|
|    -5.0|        6|
|   -23.0|        1|
|    -8.0|        2|
|   -10.0|        4|
|   -30.0|        5|
|   -11.0|        7|
|   -15.0|        1|
|    -4.0|        3|
|   -10.0|        4|
|   -23.0|        6|
|   -34.0|        7|
|   -35.0|        3|
|   -17.0|        1|
|    -1.0|        3|
+--------+---------+
only showing top 20 rows



In [46]:
#Ahora haz lo mismo para cada día pero solo con el retraso a la llegada,
# desagregado por cada aeropuerto de salida, utilizando la función pivot().

In [62]:
from pyspark.sql.functions import avg
vuelos_dia_retraso_llegada = (cleanFlightsDF.groupBy("Origin")\
                                     .pivot("DayOfWeek")\
                                     .avg("ArrDelay"))
vuelos_dia_retraso_llegada.show()

+------+------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+
|Origin|                 1|                  2|                  3|                   4|                   5|                  6|                  7|
+------+------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+
|   BGM|               5.5|  5.951219512195122| 23.097560975609756|   60.09756097560975|  13.868421052631579|              11.24| -2.813953488372093|
|   PSE|-4.416666666666667| 10.545454545454545|               -9.8|  0.6857142857142857| -1.7647058823529411|  2.088235294117647| -4.151515151515151|
|   INL|-4.606060606060606|  12.88888888888889|-1.2352941176470589|   20.08823529411765| -11.588235294117647|-13.129032258064516|  3.967741935483871|
|   MSY|3.0626750700280114|  2.519657454262359|  5.621753246753247|  2.8130290761869707|   5.1463057